# PROYECTO IA: Agente IA generativa para responder cuestiones sobre karate para un club deportivo.

## 1. Instalación de librerias

In [2]:
!pip install langchain pypdf openai langchain_experimental langchain_openai faiss-cpu tiktoken

!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl langchain sentence-transformers faiss-gpu



In [3]:
!pip install InstructorEmbedding

!pip install sentence-transformers==2.2.2


  Using cached sentence_transformers-2.2.2-py3-none-any.whl
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.0.1
    Uninstalling sentence-transformers-3.0.1:
      Successfully uninstalled sentence-transformers-3.0.1


In [4]:
!pip install --upgrade --quiet wikipedia

!pip install -U duckduckgo-search



In [5]:
!pip install langgraph

## 2. Carga de librerias.

In [6]:
import torch
import transformers
import os
import getpass
import operator
import json

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import BaseMessage, FunctionMessage, HumanMessage
from langchain.docstore.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from langchain_core.utils.function_calling import convert_to_openai_function
from langgraph.graph import StateGraph, END

from operator import itemgetter

from typing import TypedDict, Annotated, Sequence




## 3. Comprobación conexión GPU.

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


## 4. IMPLEMENTACIÓN DE UN RAG.

El propósito de estos pasos es procesar los documentos PDF, dividirlos en fragmentos manejables y luego crear un almacén de vectores de búsqueda utilizando embeddings.

1. Cargar archivos PDF:

El código itera sobre cada URL en la lista pdf_urls.
Para cada URL, utiliza PyPDFLoader para cargar el archivo PDF.
Luego extrae las páginas de cada PDF y las agrega a la lista all_pages.

2. Combinar el texto de todas las páginas:

El código combina el contenido de texto de todas las páginas en un solo documento.
Este texto combinado se utiliza para crear un único objeto Document.

3. Configurar el divisor de texto:

El código configura un RecursiveCharacterTextSplitter con un tamaño de chunk y un overlap.

4. Dividir el contenido combinado:

El documento de texto combinado se divide en fragmentos basados en el tamaño de fragmento y la superposición configurados.
Los fragmentos resultantes se almacenan en la lista docs.

5. Imprimir número de fragmentos y fragmento de muestra:

El código imprime el número de fragmentos creados.
También imprime el contenido de uno de los fragmentos para comprobar que esté todo correcto.

6. Crear embeddings:

El código inicializa un modelo de embeddings HuggingFaceInstructEmbeddings utilizando el nombre de modelo especificado (hkunlp/instructor-xl).

Luego crea un almacén de vectores FAISS a partir de los fragmentos de documentos y sus embeddings.

7. Crear recuperador:

El código convierte el almacén de vectores FAISS en un retriever para recuperar fragmentos de texto relevantes en base a consultas.


In [8]:
# List of PDF URLs
pdf_urls = [
    'https://www.federacioncylkarate.com/wp-content/uploads/2021/07/parakarate_-_normativa_gradosdb4e.pdf',
    'https://www.federacioncylkarate.com/wp-content/uploads/2021/07/normativa_grados_fcylk_enero_2012.pdf',
    'https://www.federacioncylkarate.com/wp-content/uploads/2024/01/RFEK-NORMATIVA-COMP-CAT.INFERIORES-2024.pdf',
    'https://www.federacioncylkarate.com/wp-content/uploads/2024/01/RFEK-NORMATIVA-COMP-KUMITE-2024.pdf',
    'https://www.federacioncylkarate.com/wp-content/uploads/2024/04/Reglamento_Competicion_Para-Karate_RFEK_2024_1.0.pdf',
    'https://www.federacioncylkarate.com/wp-content/uploads/2024/01/RFEK-NORMATIVA-COMP-KATA-2024.pdf'
]

# Initialize a list to hold the pages content of each PDF
all_pages = []

for url in pdf_urls:
    # Load the PDF file
    loader = PyPDFLoader(url)
    pages = loader.load()

    # Append the pages to the list
    all_pages.extend(pages)

# Combine all the pages text into a single document
combined_text = "\n".join([page.page_content for page in all_pages])
combined_text

documents = [Document(page_content=combined_text)]

# Configure the text splitter
chunk_size = 100
chunk_overlap = 5
r_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

# Split the combined content into chunks
docs = r_splitter.split_documents(documents)

# Get the number of resulting chunks
print(len(docs))

page = docs[10]
print(page)

embeddings = HuggingFaceInstructEmbeddings(model_name = "hkunlp/instructor-xl")

faiss_vectorstore = FAISS.from_documents(
    documents=docs,
    embedding=embeddings
)

retriever = faiss_vectorstore.as_retriever()



2556
page_content='NORMATIVA  NACIONAL  DE GRADOS  DE PARA -KARATE        - RFEK  Y DA 
 
 
Aprobado en Junta Directiva RFEK y DA  
10-Abril 2018  
3 
  
CRITER IOS DE VALORACIÓN'
load INSTRUCTOR_Transformer
max_seq_length  512


8. Implementación del prompt del RAG.

In [9]:
RAG_PROMPT = """\
Use the following context to answer the user's query.
If you cannot answer the question, please respond with 'Esta respuesta no la sé.'.

Question:
{question}

Context:
{context}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

9. Implementación de cadena de generación aumentada por recuperación (RAG) utilizando el modelo de chat de OpenAI, gpt-3.5-turbo. La cadena se configura para manejar consultas del usuario mediante la recuperación de contexto relevante y la generación de respuestas basadas en ese contexto.

    a. Configurar la clave de OpenAI.

    b. Inicializar el modelo.

    c. Definir la cadena de RAG.

      - Asignar contexto y pregunta: Se toma la pregunta del usuario (itemgetter("question")) y se utiliza con el retriever para obtener el contexto relevante. Se asigna a la clave "context".

      - Asignar el contexto a un objeto RunnablePassthrough: Asegura que el contexto obtenido en el paso anterior se mantenga y esté disponible para el siguiente paso.

      - Generar respuesta usando el prompt y el modelo de OpenAI: Los valores de "context" y "question" se utilizan para formatear el objeto rag_prompt, que luego se pasa al modelo de chat de OpenAI para generar la respuesta. La respuesta se almacena en la clave "response" y el contexto se mantiene en la clave "context".

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")
retrieval_augmented_generation_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")})



OpenAI API Key:··········


10. Invocación asincrónica de la cadena RAG configurada anteriormente para obtener una respuesta a la pregunta proporcionada.
Debido a las limitaciones de pago de OpenAI da un error hasta que se añada una ApiKey válida.

In [11]:
await retrieval_augmented_generation_chain.ainvoke({"question" : "Que es el karate?"})

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

## 5. IMPLEMENTACIÓN DE UN TOOL BELT

Configuración de una serie de herramientas (tool_belt) que se integran con un modelo de chat de OpenAI. Estas herramientas permiten al modelo de chat realizar búsquedas en DuckDuckGo y consultas en Wikipedia para proporcionar respuestas más precisas y contextuales a las preguntas del usuario.

- Configuración de herramientas: Se configuran herramientas para realizar búsquedas en DuckDuckGo y consultas en Wikipedia.

- Ejecutor de herramientas: Se crea un ejecutor para manejar estas herramientas.

- Modelo de Chat: Se inicializa un modelo de chat de OpenAI con una temperatura específica.

- Conversión de funciones: Las herramientas se convierten en funciones compatibles con OpenAI.

- Vinculación de funciones: Se vinculan estas funciones al modelo de chat.

In [12]:
tool_belt = [
    DuckDuckGoSearchRun(),
    WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
]
tool_executor = ToolExecutor(tool_belt)

model = ChatOpenAI(temperature=0.5)

functions = [convert_to_openai_function(t) for t in tool_belt]
model = model.bind_functions(functions)

## 6. IMPLEMENTACIÓN DE LA CLASE AGENTSTATE

- AgentState: Define la estructura del estado que contiene los mensajes.

- call_model: Invoca un modelo de lenguaje con los mensajes y devuelve la respuesta.

- call_tool: Invoca una herramienta basada en el último mensaje recibido, procesa la respuesta de la herramienta y devuelve un nuevo mensaje con el resultado.


Este diseño permite que un agente maneje tanto la generación de lenguaje natural como la ejecución de acciones específicas utilizando herramientas definidas.

In [13]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

## 7. IMPLEMENTACIÓN DEL FLUJO DE TRABAJO.

Define un grafo de estado con nodos para el modelo y la herramienta, y establece un punto de entrada.

In [14]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)

workflow.set_entry_point("agent")

Agregar transiciones condicionales entre nodos en el flujo de trabajo y compilar el flujo de trabajo en una aplicación.

In [15]:
def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

workflow.add_edge("action", "agent")

app = workflow.compile()

## 8. INVOCAR EL FLUJO DE TRABAJO.

In [16]:
inputs = {"messages" : [HumanMessage(content="¿Edad minima cinto negro?")]}

app.invoke(inputs)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}